In [21]:
'''
letter bigram identification
- i DONT want to use punctuation (will have to skip it)
- i WILL want to count spaces
- import them, make them into a string
- 
'''

'\nletter bigram identification\n- i DONT want to use punctuation (will have to skip it)\n- i WILL want to count spaces\n- import them, make them into a string\n- \n'

In [22]:
from string import punctuation
import re

In [23]:
# reading

feng = open("../Data/Input/LangId.train.English", 'r', encoding='utf8')
ffra = open("../Data/Input/LangId.train.French", 'r', encoding='utf8')
fita = open("../Data/Input/LangId.train.Italian", 'r', encoding='utf8')

eng_string = feng.read(); eng_string = eng_string.lower();
fra_string = ffra.read(); fra_string = fra_string.lower();
ita_string = fita.read(); ita_string = ita_string.lower();

feng.close();
ffra.close();
fita.close();

# print(ita_string)

In [24]:
print(punctuation)

# rule = r' \n\.,' # take out everything but apostrophes and single spaces

# re.sub()
    
# cleaning
def clean_text(data):
    # for i in range(len(data)):
    #     if data[i] == '\'':
    #         continue

    # lowering
    data = data.lower()

    # getting rid of punctuation and numbers
    for char in punctuation:
        if char == '\'': 
            continue
        data = data.replace(char, ' ')
    for num in '1234567890':
        data = data.replace(num, '')

    # whitespace
    data = data.strip()
    # data = data.replace("\n", " ")
    # re.sub(" +", " ", data)
    data = data.replace("   ", " ")
    data = data.replace("  ", " ")
    data = data.replace(" \'", "\'")
    data = data.replace("\' ", "\'")

    return data



                               
'''
if i is a space and i+1 is a comma/apostrophe/semicolon/, continue and do nothing else
if i+1 is . and i+2 is \n, change . to </s> and \n to <s> as long as there's a letter after it
    if i is </s> and i+1 is \n, change \n to <s> as long as there's a letter on the next line
if i find "' - s", add '\'',s to the dict and move i forward far enough
THEY USED A DASH TO MARK POSSESSIVES

i might not actually care about punctuation other than apostrophes
    i want english to have a fuckton of 's and s'
    i want french to have a fuckton of j' and t'
    i want spanish to literally have 0 of them
    italian should have l' but so should french
'''
if "\"" in punctuation:
    print("found???")
print(clean_text(eng_string[:2000]))
# print("\n");
# # print(clean_text(esp_string[:2000]))
# # print("\n");
# print(clean_text(ita_string[:2000]))
# print("\n");
# print(clean_text(fra_string[:2000]))
print("")

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
found???
approval of the minutes of the previous sitting 
the minutes of yesterday's sitting have been distributed 
are there any comments 
mr president on monday i made a point of order about president nicole fontaine's reported comments in the british press regarding her recent visit with her majesty queen elizabeth ii 
a british labour member of this house mr miller repeated what were purported to be the queen's remarks not once but three times on monday on tuesday and on wednesday 
he sought to drag the queen into a political controversy and to use her name to score cheap political points 
mr president it may be that president fontaine was unfamiliar with the british rules of protocol which obtain in such matters but mr miller has no such excuse 
he knows perfectly well that ours is a non political head of state 
he knows perfectly well that for nearly fifty years she has scrupulously avoided engaging in controversial political issues 
he knows perf

In [25]:
'''
making bigrams and unigrams together. honestly this shouldn't work
because the unigram then won't add the number of the last unigram to 
it but the results were good
'''
def make_bigram(data):
    bigram_count = {} # make a dictionary that maps bigrams to their counts
    unigram_count = {} # make a dictionary that maps bigrams to their counts

    # CLEANING:

    data = clean_text(data)

    # MAKING nGRAMS
    for i in range(len(data) - 1):
        # skip line breaks
        # if (data[i] == '\n' or data[i+1] == '\n') : continue
        ''' 
        change the mf to be the sentence end token instead of \n. 
        every line is a sentence: 
            every beginning of a line is <s>
            every end of line/period/sentence is </s>
        all the punctuation has to be switched out 
        you ain't even have to do this until way later bruh/
        '''

        if (data[i], data[i+1]) in bigram_count:            # if it's already counted, increment
            bigram_count[(data[i], data[i + 1])] += 1
        else:
            bigram_count[(data[i], data[i + 1])] = 1        # if it's new, put it in as 1

        if data[i] in unigram_count:            # if it's already counted, increment
            unigram_count[(data[i])] += 1
        else:
            unigram_count[(data[i])] = 1        # if it's new, put it in as 1

    return (bigram_count, unigram_count)

# print("english \n", make_bigram(eng_string)[0], "\n", make_bigram(eng_string)[1])
# print("\n")
# print("french  \n", make_bigram(fra_string)[0], "\n", make_bigram(fra_string)[1])
# print("\n")
# print("italian \n", make_bigram(ita_string)[0], "\n", make_bigram(ita_string)[1])
# print("\n")
# print("spanish ", make_bigram(esp_string))


In [26]:
from math import log
def score_line(line, bigram, unigram):
    '''
    scores a line (could change this to word) at a time 
    data should be the input data
    bigram and unigram come from training
    '''
    score = 0
    n = 1
    V = len(unigram)
    for i in range(len(line) - 1) :
        denom = V 
        numer = n
        if line[i] in unigram.keys():
            denom += unigram[line[i]]
        if (line[i], line[i+1]) in bigram.keys():
            numer += bigram[(line[i], line[i+1])]
        score += log(numer/denom)
    return score    
        
    

In [27]:
ita_grams = make_bigram(ita_string)
eng_grams = make_bigram(eng_string)
fra_grams = make_bigram(fra_string)

def score(data) :
    ans = ''
    # scores = {"English" : 0, "Italian" : 0, "French" : 0}
    data = data.split("\n")
    for i in range(len(data)) :
        '''
        compare against all 3 bigrams and unigrams, add the highest score to the string
        '''
        line = data[i]
        ans += f'{i + 1} '
        
        scores = {}

        ita_score = score_line(line, ita_grams[0], ita_grams[1])
        scores[ita_score] = "Italian"

        eng_score = score_line(line, eng_grams[0], eng_grams[1])
        scores[eng_score] = "English"

        fra_score = score_line(line, fra_grams[0], fra_grams[1])
        scores[fra_score] = "French"
        
        ans += scores[max(scores.keys())]

        if i < 299: ans += "\n"
        
    return ans


In [28]:
# runs the whole thing
def main(test_file) :
    test = open(test_file, "r")
    test_str = test.read()
    test.close()
    ans = score(clean_text(test_str))
    out = open("../Data/Output/letterLangId.out", "w")
    out.write(ans)
    out.close()

In [29]:
main("../Data/Validation/LangId.test")

In [30]:
# compare output and expected
def comp(out_file, sol_file) :
    out = open(out_file, "r")
    sol = open(sol_file, "r")
    num = 0
    out = out.readlines()
    sol = sol.readlines()
    for i in range(len(out)) :
        if (out[i] == sol[i]): num += 1
        
    return (num/len(out))*100

print(comp("../Data/Output/letterLangId.out", "../Data/Validation/labels.sol"))

99.0
